<a href="https://colab.research.google.com/github/keywoong/deeplearning_with_python/blob/main/3_2_Getting_started_with_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.4 Classifying movie reviews : a binary classification example
## 3.4.1 The IMDB dataset
#### IMDB dataset : 인터넷 영화 데이터베이스에서 얻은 50000개의 리뷰, 25000개는 training set, 25000개는 test set(평가의 내용은 50%는 부정평가, 50%는 긍정평가로 구성되어 있다)

In [ ]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

#### 로드된 데이터들의 형태는 List 형태의 데이터로 확인된다.

In [ ]:
print(train_data.dtype)

object


#### num_words=10000를 통해 training 데이터에서 가장 자주 발생하는 단어 10000개만 유지하고 나머지 rare한 단어들은 버린다. 
#### 이를 통해 manageable한 크기의 벡터 데이터로 학습할 수 있다.

In [ ]:
# 0 is negative, 1 is positive
print(train_data[0])
print(train_labels[0])
# word vector로 감상평을 나열
# 1 이므로 이 영화에 대하여 부정적인 감상평

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


In [ ]:
# 자주 나오는 단어를 10000개로 지정했기 때문에 어떤 단어의 index도 10000을 넘지 못한다.
print(max([max(sequence) for sequence in train_data]))

9999


#### 로드된 데이터의 decoding을 위하여 아래와 같이 코드를 실행할 수 있다.

In [ ]:

word_index = imdb.get_word_index() # dictionary mapping words to an integer index
reverse_word_index = dict([(value, key) for (key,value) in word_index.items()]) # mapping integer indices to words
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])
# 0,1 및 2는 'padding', 'start of sequence (sos)', 'unknown'에 대해 예약 된 인덱스이기 때문에 인덱스는 3만큼 오프셋.

print(decoded_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

## 3.4.2 Preparing the data
#### 로드된 데이터는 정수꼴 list 형태로, 바로 네트워크에 넣지 못한다. 데이터들을 tensor로 바꾸어 주어야 한다. 바꾸는 방법은 크게 2가지가 있다.
#### 1. list는 각각마다 길이가 다르므로, 고정된 길이의 텐서에 넣는다. integer tensor of shape(samples, word_indices)
#### 2. 데이터들로 one-hot 코딩을 한다. 예를 들어 [3,5] 와 같은 데이터는 10000차원 벡터에서 4번째, 6번째 (0부터 시작이므로)에만 1이라는 값을 써주고 나머지 9998개의 값은 0으로 채운다. 이렇게 변환시키면 실수 벡터 데이터를 처리할 수 있는 Dense 계층의 첫번째 층으로 사용할 수 있다.

#### 이 장에서는 후자의 방법인 one-hot-encoding 으로 진행한다.

In [ ]:
# 정수 형태의 시퀀스를 원핫코딩로 인코딩
import numpy as np

def vectorize_sequences (sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    # matrix의 모든 값을 0으로 변환
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
        # 특정 지수에만 1을 넣어줌(원핫코딩)
    return results

x_train = vectorize_sequences(train_data) # training data를 벡터화
x_test = vectorize_sequences(test_data) # test data를 벡터화


y_train = np.asarray(train_labels).astype('int')
y_test = np.asarray(test_labels).astype('float32')

In [ ]:
print(x_train[0].dtype)
print(x_train[0]) # 영화 감상평을 원핫 인코딩으로 한 값
print(y_train[0]) # 첫번째 감상평이 부정적인지, 긍정적인지를 알려주는 값

'''
float64
[0. 1. 1. ... 0. 0. 0.] , wordvector
1, 긍정
'''

float64
[0. 1. 1. ... 0. 0. 0.]
1


'\nfloat64\n[0. 1. 1. ... 0. 0. 0.] , wordvector\n1, 긍정\n'

## 3.4.3 Building your network
#### 입력 데이터는 벡터고, 출력 데이터는 스칼라 (0과 1), relu함수를 가지는 완전연결 계층 모델이 네트워크 구성에 적합하다.
#### 여기서 각 dense 계층에 전달되는 16 개의 인수는 각 계층의 은닉 유닛(hidden units)의 수가 된다. 또한 은닉
유닛 유닛의 수는 해당 계층의 표현 공간(representation space)의 차원이 된다.
#### relu 활성화 함수가 있는 dense layer은 아래와 같은 텐서 함수를 가진다.
#### -> output = relu (dot ( W, input ) + b)
#### 은닉층의 개수가 16개이면 가중치 행렬은 shape(input_dimension, 16)
#### W와의 dot 연산으로 입력 데이터를 16차원의 표현 space로 만들어줌. b(bias)를 더하고 relu 연산을 한다.
#### -> layer은 층을 말하고, unit은 그 layer안에 있는 node의 개수이다. 한 node당 하나의 변수를 맡기 때문에 node의 개수가 차원이다.
#### 은닉층의 개수가 더 많아지면 더 복잡한 representation을 배울 수 있지만 너무 많아지면 계산을 너무 많이 하게 되어 원하지 않는 패턴을 얻게 될 수 있다.
#### 따라서 Dense layer을 만들 때는 2가지를 고려한다. "얼마나 층을 만들것인가" & "각 층에 은닉층을 얼마나 넣을 것인가"
#### 

In [ ]:
# 모델 정의하기
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation = 'relu', input_shape = (10000,))) #num_words가 10000이기 때문에(?)
model.add(layers.Dense(16,activation = 'relu'))
model.add(layers.Dense(10,activation = 'sigmoid'))

### 왜 활성화함수를 사용해야 하는 건인가?
#### ReLU와 같은 활성화 함수가 없으면 '곱하기'나 '덧셈'과 같은 선형변환(아핀 변환)밖에 하지 못한다.
#### 레이어의 가설공간(representation)은 입력 데이터를 16 차원 공간으로 가능한 모든 선형 변환의 집합이다.
#### 선형 층만 계속 존재한다면 선형변환 operation밖에 존재하지 않기 때문에 representation이 확장할 수 없다.
#### 보다 더 풍부한 형태의 표현방식을 원한다면 비선형 함수를 사용해야 한다.

In [ ]:
# Compiling the model
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Keras 내 패키지로 존재하기 때문에 optimzier, loss funtion, metrics 모두 string 형태로 쓴다.

### model compiling
#### 위의 사례는 영화에 대하여 긍정이냐, 부정이냐를 반별하므로 binary Classification이라고 할 수 있다. 따라서 적절한 목적함수는 binary_crossentropy이다.
#### 다른 목적함수로 mean_squared_error를 사용할 수 있다고 하지만, crossentropy는 출력값이 0과 1 사이인 확률인 경우에는 최적의 선택이라고 한다.
#### Crossentropy를 통해 실측 분포와 예측 사이의 거리를 측정할 수 있다.

In [ ]:
# optimizer에 대한 변수를 알기 위한 소스코드
from keras import optimizers

model.compile(optimizer = optimizers.RMSprop(lr = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# loss 함수와 metrics 변수를 알기 위한 소스코드
from keras import losses
from keras import metrics

model.compile(optimizer = optimizers.RMSprop(lr = 0.001), loss = losses.binary_crossentropy, metrics = [metrics.binary_accuracy])

## 3.4.4 Validating your approach
### 검증의 시간
#### 데이터 모델의 정확도를 측정하기 위해서, 기존의 training 데이터로부터 10000개 정도의 샘플을 뽑아 평가 set로 만든다.

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]


model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
print(partial_x_train.shape)
print(partial_y_train.shape)
print(x_val.shape)
print(y_val.shape)
history = model.fit(partial_x_train, partial_y_train, epochs = 20, batch_size = 512, validation_data = (x_val, y_val))
print(partial_x_train.shape)

(15000, 10000)
(15000,)
(10000, 10000)
(10000,)
Epoch 1/20


ValueError: in user code:

    C:\Users\Key woong Bae\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\Key woong Bae\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Key woong Bae\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Key woong Bae\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Key woong Bae\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\